In [2]:
import fecfile
import pandas as pd
import datetime
import re

Time the notebook

In [3]:
start = datetime.datetime.now()

Set some viewing options

In [4]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 500)

Read dataframe of filings

In [5]:
filings = (
    pd.read_csv("../data/filings.csv")
)

filings.head(3)

,committee_id,filing_id,cycle,form_type,date_filed,date_coverage_to,date_coverage_from,report_title,report_period,fec_uri,paper,amended,amended_uri,is_amendment,original_filing,original_uri,committee_type,contributions_total,cash_on_hand,disbursements_total,receipts_total
0,C00401224,1190211,2016,F3,2017-11-03,2016-11-28,2016-10-20,POST-GENERAL,PG,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1190211/,False,False,NaN,True,1132265.0,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1132265/,W,69306148.22,24023158.64,70473625.32,70884967.84
1,C00401224,1167570,2016,F3,2017-06-27,2016-10-19,2016-10-01,PRE-GENERAL,PREG,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1167570/,False,False,NaN,True,1118321.0,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1118321/,W,38988748.30,23611816.12,36653226.26,41086481.96
2,C00401224,1166534,2016,F3,2017-06-19,2016-12-31,2016-11-29,YEAR-END,YE,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1166534/,False,False,NaN,True,1144458.0,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1144458/,W,14665549.11,24071137.23,15378810.13,15426788.72


Get only the midyear

In [6]:
midyear = (
    filings
#     .loc[
#         lambda x: x['report_period'].str.contains("M", na=False) # if I need monthlies
#     ]
    .loc[
        lambda x: x["report_title"].str.contains("MID-YEAR", na = False)
    ]
    # remove filings that have been superceded by subsequent filings
    .loc[
        lambda x: x['amended'] == False
    ]
    [["filing_id",
        "date_filed",
        "date_coverage_to",
        "date_coverage_from",
        "report_title",
        "report_period",
        "fec_uri",
         "cycle"
     ]]
)

midyear.head(3)

,filing_id,date_filed,date_coverage_to,date_coverage_from,report_title,report_period,fec_uri,cycle
28,1034155,2015-11-23,2015-06-30,2015-06-23,MID-YEAR,NaN,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1034155/,2016


## Parse all the fec files with the filing list

### But first, more convenience functions..

This function, when given a filing ID, returns only the earmarked contributions from individual donors to candidates:

In [7]:
def extract_contributions(filing_id):
    filing = fecfile.from_file(f"../data/filings/{filing_id}.fec")
    meta = filing['filing']
    
    # get only schedule A
    schedule_a = pd.DataFrame(filing["itemizations"]["Schedule A"])
    # If I need to access the summary data
    #summary = pd.DataFrame(filing['summary'])
    
    # remove time zone
    schedule_a["contribution_date"] = schedule_a["contribution_date"].dt.tz_localize(None)
        
    return (
        schedule_a
        # Extract only individual contributions
        .loc[lambda df: df["entity_type"] == "IND"]
        # Remove memo lines
        .loc[lambda df: df["memo_code"] == ""]
        # Remove donations that are not for a specific candidate
        .loc[ lambda df: df["contribution_purpose_descrip"] == "Earmark" ]
        .assign(
            filing_id = str(filing_id),
        )
        [[
            "entity_type",
            "filer_committee_id_number",
            "filing_id",
            "transaction_id",
            "contribution_date",
            "contribution_amount",
            "contribution_aggregate",
            "contributor_organization_name",
            "contributor_first_name",
            "contributor_last_name",
            "contributor_street_1",
            "contributor_street_2",
            "contributor_state",
            "contributor_zip_code",
            "contributor_state",
            "contributor_employer",
            "contributor_occupation",
            "contribution_purpose_descrip",
            "memo_text_description"
        ]]
    )

Create a unique ID out of first name, last name and 5-digit ZIP code

In [8]:
def make_donor_ids(df):
    return (
        df
        .assign(
            donor_id = lambda df: (
                df
                .assign(
                    
                    zip5 = lambda df: (
                        df["contributor_zip_code"]
                        .fillna("-----")
                        .str.slice(0, 5)
                    )
                )
                [[
                    "contributor_first_name",
                    "contributor_last_name",
                    "zip5",
                ]]
                .apply(lambda x: (
                    x
                    .fillna("")
                    .astype(str)
                    # Remove periods, commas, extra whitespace
                    .str.replace(r"[\.,\s]+", " ")
                    .str.strip()
                    # Convert everything to upper-case
                    .str.upper()
                ))
                .apply("|".join, axis = 1)
            )
        )
    )

Add a donor name column

In [9]:
def extract_committee_id(df):
    return (
        df
        .assign(
            committee_id = lambda df: df["memo_text_description"].str.extract(r"\((C\d{8})\)", expand = False),
            actblue_committee_name = lambda df: (
                df
                ["memo_text_description"]
                .str.extract(r"^Earmarked for (.*)?\(C\d{8}\)", expand = False)
                .str.strip()
            ),
        )
    )

Concatenate all the filings data into one big DataFrame, and get individual contributions only

In [10]:
all_actblue = (
    pd
    .concat(
        [ extract_contributions(e) for e in midyear['filing_id'].tolist() ]
    )
    .pipe( make_donor_ids )
    .pipe( extract_committee_id )
    
)

all_actblue.head()

,entity_type,filer_committee_id_number,filing_id,transaction_id,contribution_date,contribution_amount,contribution_aggregate,contributor_organization_name,contributor_first_name,contributor_last_name,contributor_street_1,contributor_street_2,contributor_state,contributor_zip_code,contributor_state,contributor_employer,contributor_occupation,contribution_purpose_descrip,memo_text_description,donor_id,committee_id,actblue_committee_name
17,IND,C00401224,1034155,SA11AI_28202360,2015-06-29,5.0,35.0,,HOMA,A. HASHEMI,22 CLINTON AVE 9,,CT,06901,CT,NOT EMPLOYED,NOT EMPLOYED,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),HOMA|A HASHEMI|06901,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE
18,IND,C00401224,1034155,SA11AI_28249728,2015-06-30,5.0,35.0,,HOMA,A. HASHEMI,22 CLINTON AVE 9,,CT,06901,CT,NOT EMPLOYED,NOT EMPLOYED,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),HOMA|A HASHEMI|06901,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE
19,IND,C00401224,1034155,SA11AI_28301009,2015-06-30,10.0,35.0,,BARBARA,A. HUBBARD,89 BREEZEWOOD ROAD,,VA,24078,VA,SELF,READING TUTOR,Earmark,Earmarked for DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE (C00042366),BARBARA|A HUBBARD|24078,C00042366,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE
20,IND,C00401224,1034155,SA11AI_28108486,2015-06-27,25.0,25.0,,SHERI,A. OLSON,4008 SW ARROYO CT,,WA,98146,WA,SELF,ARCHITECT,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),SHERI|A OLSON|98146,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE
22,IND,C00401224,1034155,SA11AI_28091589,2015-06-27,15.0,30.0,,MARY,A. TODD,24701 NE 228TH CIRCLE,,WA,98604,WA,KAISER PERMANENTE,NURSE PRACTITIONER,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),MARY|A TODD|98604,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE


In [11]:
len(all_actblue)

277198

Note: Some committee IDs are either missing or cut off.

In [12]:
(
    all_actblue
    .loc[lambda df: df["actblue_committee_name"].isnull()]
    ["memo_text_description"]
    .value_counts()
)

Earmarked for LON JOHNSON FOR CONGRESS ()                                                               32
Earmarked for CHC BOLD PAC/COMMITTEE FOR HISPANIC CAUSES BUILDING OUR LEADERSHIP DIVERSITY (C0036553     5
Earmarked for ERIC LYNN FOR CONGRESS ()                                                                  2
Name: memo_text_description, dtype: int64

Note: In some cases, the same `committee_id` is associated with more than one committee name in the earmarks. We need to be careful with how we handle this.

In [13]:
(
    all_actblue
    .groupby(["committee_id"])
    ["actblue_committee_name"]
    .nunique()
    .max()
)

2

In [14]:
(
    all_actblue
    .loc[lambda df: df["committee_id"].isin(
        all_actblue
        .groupby(["committee_id"])
        ["actblue_committee_name"]
        .nunique()
        .loc[lambda x: x > 1]
        .index
    )]
    [[
        "committee_id",
        "actblue_committee_name"
    ]]
    .drop_duplicates()
    .sort_values([ "committee_id", "actblue_committee_name" ])
)

,committee_id,actblue_committee_name
203181,C00010033,MONTANA DEMOCRATIC LEGISLATIVE CAMPAIGN COMMITTEE
30,C00010033,MONTANA STATE DEMOCRATIC CENTRAL COMMITTEE - FEDERAL ACCOUNT
26287,C00010603,DEMOCRATIC NATIONAL COMMITTEE
208148,C00010603,GAY & LESBIAN LEADERSHIP COUNCIL/DNC
227,C00019331,DEMOCRATIC PARTY OF WISCONSIN - FEDERAL ACCOUNT
253192,C00019331,DEMOCRATIC PARTY OF WISCONSIN - OPERATING ACCOUNT


In [15]:
assert (
    all_actblue
    .groupby(["actblue_committee_name"])
    ["committee_id"]
    .nunique()
    .max()
) == 1

Write out before we merge with more candidate data

In [16]:
all_actblue.to_csv("../output/all_actblue.csv", index = False)

Get the candidate names using commitee IDs

In [17]:
candidates = pd.read_csv("../data/candidates.csv").rename(
    columns = {
        "Committee ID": "committee_id",
        "Committee Name": "committee_name",
        "Candidate Name": "candidate_name"
    }
)

candidates.head(3)

,candidate_name,committee_name,Candidate ID,committee_id
0,Cory Booker,Cory 2020,P00009795,C00695510
1,Kamala Harris,Kamala Harris for the People,P00009423,C00694455
2,Joe Biden,Biden for President,P80000722,C00703975


Make dummy candidate DataFrame

In [18]:
dummy_candidates = (
    all_actblue
    [[
        "committee_id",
        "actblue_committee_name",
    ]]
    .rename(columns = { "actblue_committee_name": "committee_name" })
    .drop_duplicates(subset = [ "committee_id" ])
    .dropna(subset = [ "committee_id" ])
    .assign(
        candidate_name = lambda df: df["committee_name"]
    )   
)

dummy_candidates.head()

,committee_id,committee_name,candidate_name
17,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE
19,C00042366,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE
24,C00577130,BERNIE 2016,BERNIE 2016
25,C00458000,PROGRESSIVE CHANGE CAMPAIGN COMMITTEE,PROGRESSIVE CHANGE CAMPAIGN COMMITTEE
29,C00443689,PEOPLE FOR BEN,PEOPLE FOR BEN


Merge with candidates to get names

In [19]:
with_candidates = (
    all_actblue
    .merge(
        dummy_candidates, # CHANGE TO `candidates`
        on = "committee_id",
        how = "inner", #CHANGE TO LEFT
        validate = "m:1"
    )
)


# Visually inspect to make sure committee names and candidate names match
(
    with_candidates
    [['committee_name', 'actblue_committee_name', 'committee_id']]
    .drop_duplicates()
)

,committee_name,actblue_committee_name,committee_id
0,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,C00000935
59412,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,C00042366
79045,BERNIE 2016,BERNIE 2016,C00577130
183944,PROGRESSIVE CHANGE CAMPAIGN COMMITTEE,PROGRESSIVE CHANGE CAMPAIGN COMMITTEE,C00458000
187196,PEOPLE FOR BEN,PEOPLE FOR BEN,C00443689
187239,MONTANA STATE DEMOCRATIC CENTRAL COMMITTEE - FEDERAL ACCOUNT,MONTANA STATE DEMOCRATIC CENTRAL COMMITTEE - FEDERAL ACCOUNT,C00010033
187380,MONTANA STATE DEMOCRATIC CENTRAL COMMITTEE - FEDERAL ACCOUNT,MONTANA DEMOCRATIC LEGISLATIVE CAMPAIGN COMMITTEE,C00010033
187508,DEMOCRACY FOR AMERICA,DEMOCRACY FOR AMERICA,C00370007
188786,DONNA EDWARDS FOR SENATE,DONNA EDWARDS FOR SENATE,C00574145
191834,END CITIZENS UNITED PAC,END CITIZENS UNITED PAC,C00573261


Get the aggregate amount a donor has given to a particular candidate

In [20]:
latest_contribs = (
    with_candidates
    .sort_values('contribution_date')
    .groupby(['donor_id','candidate_name'])
    .pipe(lambda grp: pd.DataFrame({
        "latest_contribution_aggregate": grp["contribution_aggregate"].last(),
        })
    )
    .reset_index()
)

latest_contribs.sort_values(
    'latest_contribution_aggregate',
    ascending = False
).head(3)


,donor_id,candidate_name,latest_contribution_aggregate
122467,KAREN|LAWRENCE|22102,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,60604.22
99009,JEANNE|LAVINE|02421,JOE KENNEDY VICTORY FUND 2016,38800.00
113622,JONATHAN|LAVINE|02421,JOE KENNEDY VICTORY FUND 2016,38800.00


Write out

In [21]:
latest_contribs.to_csv(
    "../output/latest_contribs.csv", 
    index = False
)

Merge with latest_contrib to get latest

In [22]:
clean_actblue = with_candidates.merge(
    latest_contribs,
    on = ["donor_id", 'candidate_name'],
    how = "left"
)

Write out

In [23]:
clean_actblue.to_csv(
    "../output/clean_actblue.csv", 
    index = False)

Time out

In [24]:
end = datetime.datetime.now()

d = (end - start)

f"The notebook ran for {round(d.total_seconds() / 60, 2) } minutes"

'The notebook ran for 2.01 minutes'

---

---

---